In [198]:
import cr_interface as cri
import numpy as np
import pandas as pd
import sklearn as skl

In [215]:
#results = cri.load_results(results_dir="over")
results = cri.load_results()
metadata = cri.load_metadata()

In [216]:
results.sort(key=lambda d: d['test_accuracy'])
best_result = results[-1]
best_result['test_accuracy']

'0.86178863'

In [217]:
selected_result = best_result

In [218]:
def get_result_df(result, training=False):
    if training:
        cr_codes = map(cri.extract_cr_code, result['training_images'])
    else:
        cr_codes = map(cri.extract_cr_code, result['predictions'].keys())
        
    cr_codes = list(set(cr_codes))
    
    for i, cr_code in enumerate(cr_codes):
        cr_codes[i] = 'D00' + cr_code[3:]

    extract_dataset = lambda cr_code: cri.parse_cr_code(cr_code)[0]
    extract_pid = lambda cr_code: cri.parse_cr_code(cr_code)[1]
    extract_phase = lambda cr_code: cri.parse_cr_code(cr_code)[2]
    extract_slice = lambda cr_code: cri.parse_cr_code(cr_code)[3]

    datasets = list(map(extract_dataset, cr_codes))
    pids = list(map(extract_pid, cr_codes))
    phases = list(map(extract_phase, cr_codes))
    slices = list(map(extract_slice, cr_codes))

    extract_label = lambda cr_code: metadata[cr_code]['label']
    labels = list(map(extract_label, cr_codes))
    
    # make tri labels
    for i, label in enumerate(labels):
        if label in ['ap', 'bs', 'md']:
            labels[i] = 'in'
        
    d = dict(
            cr_code=cr_codes,
            dataset=datasets,
            pid=pids,
            phase=phases,
            slice=slices,
            label=labels,
    )
    
    if not training:
        extract_prediction = lambda cr_code: result['predictions']['testing_{}'.format(cr_code)]['prediction']
        predictions = list(map(extract_prediction, cr_codes))
        d['prediction'] = predictions
        

    return pd.DataFrame(d)

In [219]:
df = get_result_df(selected_result, training=False)
df = df.drop_duplicates()
pd.crosstab(df['label'], df['prediction'])
df

,cr_code,dataset,pid,phase,slice,label,prediction
0,D00_P00039301_P00_S05,0,39301,0,5,in,in
1,D00_P00042401_P00_S07,0,42401,0,7,in,in
2,D00_P00042001_P00_S05,0,42001,0,5,in,in
3,D00_P00037101_P00_S06,0,37101,0,6,in,in
4,D00_P00040101_P00_S14,0,40101,0,14,obs,in
5,D00_P00039401_P00_S06,0,39401,0,6,in,in
6,D00_P00039501_P00_S01,0,39501,0,1,oap,in
7,D00_P00042601_P00_S04,0,42601,0,4,in,in
8,D00_P00043101_P00_S02,0,43101,0,2,in,in
9,D00_P00043701_P00_S04,0,43701,0,4,in,in


In [222]:
df = get_result_df(selected_result, training=False)
df = df.drop_duplicates()
pd.crosstab(df['label'], df['prediction'])

precision = {} # 정밀도
recall = {} # 재현율

lp = df[['label', 'prediction']]
for label in set(df['label']):
    true_positives = lp[lambda e: (e.label == label) & (e.prediction == label)]
    truths = lp[lambda e: e.label == label]
    positives = lp[lambda e: e.prediction == label]
    
    precision[label] = len(true_positives) / len(positives)
    recall[label] = len(true_positives) / len(truths)

summary_df = pd.DataFrame(dict(
    precision=precision,
    recall=recall
))
summary_df

,precision,recall
in,0.880000,0.927711
oap,0.829787,0.795918
obs,0.791667,0.612903
